COLAB - https://colab.research.google.com/drive/14C9iAqoaMT44_Ff6feYmU6pzFLfV-2qw?authuser=0#scrollTo=jzcS_Kp3bHh5

In [1]:
from google.colab import userdata
NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! uv pip install langchain langchain_nvidia_ai_endpoints langchain-community pypdf "unstructured[all-docs]" faiss-gpu-cu12

Using Python 3.12.12 environment at: /usr
Resolved 170 packages in 937ms
Prepared 41 packages in 4.32s
Uninstalled 1 package in 1ms
Installed 41 packages in 493ms
 + backoff==2.2.1
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + deprecated==1.3.1
 + effdet==0.4.1
 + emoji==2.15.0
 + faiss-gpu-cu12==1.13.2
 + filetype==1.2.0
 + google-cloud-vision==3.11.0
 + humanfriendly==10.0
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 + langchain-nvidia-ai-endpoints==1.0.1
 + langchain-text-splitters==1.1.0
 + langdetect==1.0.9
 + marshmallow==3.26.2
 + msoffcrypto-tool==5.4.2
 + mypy-extensions==1.1.0
 + olefile==0.47
 + onnx==1.20.0
 + onnxruntime==1.23.2
 + pdf2image==1.17.0
 + pdfminer-six==20260107
 + pi-heif==1.1.1
 + pikepdf==10.1.0
 + pypandoc==1.16.2
 + pypdf==6.5.0
 + pypdfium2==5.3.0
 + python-docx==1.2.0
 + python-iso639==2025.11.16
 + python-magic==0.4.27
 + python-oxmsg==0.0.2
 + python-pptx==1.0.2
 + rapidfuzz==3.14.3
 - requests==2.32.4
 + requests==2.32.5
 + typin

In [7]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
LLM_MODEL = "nvidia/nvidia-nemotron-nano-9b-v2"
llm = ChatNVIDIA(model=LLM_MODEL, temperature=0.6, top_p=0.95, max_tokens=8192, api_key=NVIDIA_API_KEY)


/tmp/ipython-input-1458240435.py:3: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  llm = ChatNVIDIA(model=LLM_MODEL, temperature=0.6, top_p=0.95, max_tokens=8192, api_key=NVIDIA_API_KEY)


In [8]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'reasoning_content': 'Okay, the user just said "Hi". That\'s a greeting. I should respond politely. Let me make sure to acknowledge their greeting and offer help. Maybe say something like "Hello! How can I assist you today?" Keep it friendly and open-ended.'}, response_metadata={'role': 'assistant', 'reasoning_content': 'Okay, the user just said "Hi". That\'s a greeting. I should respond politely. Let me make sure to acknowledge their greeting and offer help. Maybe say something like "Hello! How can I assist you today?" Keep it friendly and open-ended.', 'content': 'Hello! How can I assist you today? 😊', 'tool_calls': [], 'token_usage': {'prompt_tokens': 14, 'total_tokens': 84, 'completion_tokens': 70, 'prompt_tokens_details': None}, 'finish_reason': 'stop', 'model_name': 'nvidia/nvidia-nemotron-nano-9b-v2'}, id='lc_run--019b9a54-84c6-72f0-b5e3-490d5ecd07d1-0', usage_metadata={'input_tokens': 14, 'output_token

In [9]:
DATA_DIR = "/content"

In [10]:
# from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, UnstructuredPDFLoader
# # Read the data
# print(f"Reading knowledge base data from {DATA_DIR}")
# # data_loader = DirectoryLoader(
# #     DATA_DIR,
# #     glob="**/*",
# #     loader_cls=PyPDFLoader,
# #     show_progress=True,
# # )

# loader = UnstructuredPDFLoader("/content/qps-ugs.pdf", show_progress=True)
# # docs = data_loader.load()


Reading knowledge base data from /content


In [11]:
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,228 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy

In [19]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# Recommended values for the 8,192 token limit of NVIDIA NeMo Retriever
# Assuming ~4 characters per token, 16,000 chars is roughly 4,000 tokens.
MAX_CHARS = 16000
SOFT_MAX = 12000

loader = UnstructuredPDFLoader(
    "/content/qps-ugs.pdf",
    mode="elements",                # Essential for identifying Titles, Tables, and Images
    strategy="auto",                # Fast for text pages, hi_res only when needed
    chunking_strategy="by_title",   # Keeps section context together
    max_characters=MAX_CHARS,       # Hard limit to stay under token window
    new_after_n_chars=SOFT_MAX,     # Preferred chunk size
    combine_text_under_n_chars=2000, # Merges tiny fragments into larger chunks
    overlap=0,  # Not needed for clean semantic chunks
    pdf_infer_table_structure=True,  # Optional: adds HTML version of tables to metadata
    # show_progress=True,
)

In [20]:

# Load the structured documents
docs = loader.load()

print(f"Total chunks created: {len(docs)}")

Total chunks created: 1640


In [21]:
# Inspect chunks
import pandas as pd
from unstructured.staging.base import elements_to_dicts

# 1. Convert the list of elements/docs into a list of dictionaries
# If you are using LangChain's 'docs', you can access the original elements if needed
# or manually create a list of dicts from the Document objects.
element_dicts = [
    {
        "page_content": doc.page_content,
        "category": doc.metadata.get("category"),
        "page_number": doc.metadata.get("page_number"),
        "filename": doc.metadata.get("filename")
    }
    for doc in docs
]

# 2. Create the Pandas DataFrame
df = pd.DataFrame(element_dicts)

# 3. View the first few rows
df.head()

,page_content,category,page_number,filename
0,"Auto-generated Date: August 18, 2025\n\nIntel®...",CompositeElement,1,qps-ugs.pdf
1,Intel Quartus Prime Standard Edition User Guid...,CompositeElement,3,qps-ugs.pdf
2,Intel Quartus Prime Standard Edition User Guid...,CompositeElement,4,qps-ugs.pdf
3,"Intel Arria® 10, Arria V, and Arria II\n\nInte...",CompositeElement,6,qps-ugs.pdf
4,Intel Quartus Prime Standard Edition User Guid...,CompositeElement,7,qps-ugs.pdf


In [22]:
len(df)

1640

In [23]:

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

client = NVIDIAEmbeddings(
  model="nvidia/llama-3.2-nemoretriever-1b-vlm-embed-v1",
  api_key=NVIDIA_API_KEY,
  truncate="NONE",
  )

embedding = client.embed_query("What is the civil caseload in South Dakota courts?")
print(embedding)



[0.01407685037702322, 0.002299611223861575, 0.004979926161468029, -0.003989155404269695, 0.036100223660469055, 0.04268902912735939, 0.009005141444504261, -0.005454862955957651, -0.02851065620779991, 0.00699287885800004, -0.014540918171405792, -0.002283183392137289, -0.010799409821629524, 0.010147376917302608, -0.009050436317920685, -0.0026588921900838614, -0.021890314295887947, -0.04144589975476265, 0.01741761714220047, 0.02275760844349861, -0.017110414803028107, -0.03585471212863922, -0.030209969729185104, 0.0491521917283535, 0.028357811272144318, -0.04464305192232132, 0.02718381956219673, -0.008959469385445118, -0.03141769766807556, -0.0032825295347720385, 0.0020062937401235104, 0.025830155238509178, -0.031007088720798492, -0.0010382018517702818, -0.004374569747596979, -0.02469269186258316, 0.019797325134277344, -0.035076722502708435, 0.02216874435544014, -0.015468867495656013, -0.028406748548150063, -0.02895757369697094, 0.004557916894555092, -0.024648088961839676, -0.02534268423914

In [24]:
from langchain_community.vectorstores import FAISS

# This will automatically call client.embed_documents() in optimized batches
vector_store = FAISS.from_documents(docs, client)



In [26]:
# Save to your Google Drive persistently
vector_store.save_local("/content/drive/My Drive/my_project/faiss_index_fpga_copilot")
print("Vector store created and saved to Drive.")

Vector store created and saved to Drive.


In [28]:
!uv pip install rich

Using Python 3.12.12 environment at: /usr
Audited 1 package in 128ms


In [29]:
from rich import print as print

In [33]:
embeddings_path = "/content/drive/My Drive/my_project/faiss_index_fpga_copilot"

In [34]:
import os
from langchain_community.vectorstores import FAISS

if os.path.exists(embeddings_path):
    # Load the vector store
    # You MUST provide the original embedding client (NVIDIAEmbeddings)
    vector_store = FAISS.load_local(
        embeddings_path,
        client,
        allow_dangerous_deserialization=True
    )
    print("Vector store loaded successfully from Drive.")
else:
    print(f"Error: No index found at {embeddings_path}. Did you save it first?")

Vector store loaded successfully from Drive.

In [30]:
# 1. Define your test question
query = "What is the primary function of the Intel Quartus Prime software?"

# 2. Perform the search
# k=3 tells it to find the top 3 matches
results = vector_store.similarity_search(query, k=3)

# 3. Print the results with metadata
for i, res in enumerate(results):
    print(f"--- Result {i+1} ---")
    print(f"Page: {res.metadata.get('page_number')}")
    print(f"Type: {res.metadata.get('category')}")
    print(f"Content: {res.page_content[:300]}...\n")

--- Result 1 ---

Page: 6

Type: CompositeElement

Content: Intel Arria® 10, Arria V, and Arria II

Intel Cyclone® 10 LP, Cyclone IV, and Cyclone V

MAX® series

The Intel Quartus Prime software GUI supports easy design entry, fast design processing, straightforward device 
programming, and integration with other industry- standard EDA tools. The user interfa...

--- Result 2 ---

...

Content: Note:

Intel replaces the following Altera tool names in the Intel Quartus Prime software:

Table 1.

Intel Quartus Prime Tool Name Updates

Altera Name

Intel Name

Qsys

Platform Designer

TimeQuest

Timing Analyzer

EyeQ

Eye Viewer

JNEye

Advanced Link Analyzer

Related Information

Migrating t...

--- Result 3 ---

Page: 1293

Type: CompositeElement

Content: Intel Quartus Prime Standard Edition User Guide: Third-party Synthesis

3

683796 | 2018.09.24

Send Feedback

1. Synopsys Synplify* Support

1.1. About Synplify Support

the Intel® Quartus® Prime software supports use of the Synopsys Synplify software design flows, methodologies, and 
techniques for...

In [32]:
results[0].__dict__.keys()

dict_keys(['id', 'metadata', 'page_content', 'type'])